In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
df=pd.read_excel('/content/EDAoutput9421.xlsx')

In [4]:
df['pH change']=abs(df['pH change'])

In [5]:
df

,DOC,Daily feed amount(Kg),pH change,Water colour,Biofloc amount(ml/L),SDR(cm),Carbon essence(Kg),Water level(cm),Water temperature(°C),Salinity(ppt),Operated HP,Average DO(ppm),Surface temperature(°C),Wind speed(m/s),Precipitation(mm/day),Surface pressure(kPa),RH(%)
0,1,1.7,0.3,6,1.8,50,15,120,29.10,16,56,5.78,27.09,3.22,0.10,100.88,71.93
1,2,2.4,0.1,8,2.0,48,15,120,28.65,16,31,5.60,27.60,3.36,0.08,100.82,71.66
2,3,3.4,0.1,6,0.5,55,12,132,28.10,16,29,5.33,27.49,3.29,0.18,100.88,73.13
3,4,4.2,0.0,6,0.5,60,11,128,28.40,16,44,5.47,27.81,3.99,2.01,100.95,76.21
4,5,5.0,0.2,6,0.5,45,10,126,28.15,16,57,5.43,27.84,3.28,3.42,100.90,76.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,96,115.0,0.1,2,7.0,20,1,138,27.30,27,201,4.57,30.45,5.35,0.47,100.27,69.13
196,97,115.0,0.2,2,6.5,22,1,140,27.20,27,220,4.50,30.27,5.22,0.29,100.33,66.85
197,98,130.0,0.2,2,5.0,20,1,145,28.00,27,246,4.45,30.92,4.74,0.16,100.30,59.30
198,99,135.0,0.2,2,5.0,22,1,145,27.65,27,262,4.52,31.13,4.44,0.83,100.36,58.00


In [6]:
x=df[['Operated HP','Surface temperature(°C) ','Daily feed amount(Kg)','Biofloc amount(ml/L)','Salinity(ppt)','SDR(cm)','DOC','Wind speed(m/s) ','Precipitation(mm/day) ','Surface pressure(kPa) ','RH(%) ']]
y = df['Average DO(ppm)']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [7]:

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1536      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 166,401
Trainable params: 166,401
Non-trainable params: 0
_________________________________________________________________


In [8]:
model=NN_model.fit(x_train,y_train, epochs=500, batch_size=32, validation_split = 0.2)
model

Epoch 1/500
4/4 [==============================] - 1s 180ms/step - loss: 2.8357 - mean_absolute_error: 2.8357 - val_loss: 0.8284 - val_mean_absolute_error: 0.8284
Epoch 2/500
4/4 [==============================] - 0s 19ms/step - loss: 0.5199 - mean_absolute_error: 0.5199 - val_loss: 0.6386 - val_mean_absolute_error: 0.6386
Epoch 3/500
4/4 [==============================] - 0s 18ms/step - loss: 0.6450 - mean_absolute_error: 0.6450 - val_loss: 0.3225 - val_mean_absolute_error: 0.3225
Epoch 4/500
4/4 [==============================] - 0s 17ms/step - loss: 0.4124 - mean_absolute_error: 0.4124 - val_loss: 0.6926 - val_mean_absolute_error: 0.6926
Epoch 5/500
4/4 [==============================] - 0s 22ms/step - loss: 0.5074 - mean_absolute_error: 0.5074 - val_loss: 0.3379 - val_mean_absolute_error: 0.3379
Epoch 6/500
4/4 [==============================] - 0s 20ms/step - loss: 0.3483 - mean_absolute_error: 0.3483 - val_loss: 0.2478 - val_mean_absolute_error: 0.2478
Epoch 7/500
4/4 [==========

In [9]:
y_pred=NN_model.predict(x_test)

In [10]:
y_pred=y_pred.round(1)

In [11]:
y_test = pd.DataFrame(y_test)
y_test

,Average DO(ppm)
95,4.45
15,5.70
30,5.82
158,4.80
128,5.58
115,5.63
69,4.97
170,4.73
174,4.73
45,4.52


In [12]:
y_pred=y_pred.reshape(50,1)
def score(y_pred, y_test):
    error = np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean() ** 0.5
    score = 1 - error
    return score
score(y_pred, y_test)

Average DO(ppm)    0.977115
dtype: float64